# Capitulo 10: Pensamiento Extendido (Extended Thinking)

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu API key y establecer las funciones auxiliares.

**Nota:** Los prompts de ejemplo se mantienen en ingles ya que las tecnicas de ingenieria de prompts son independientes del idioma y Claude responde de manera mas predecible en ingles.

In [ ]:
!pip install anthropic

import re
import anthropic
import sys, os

# Asegurar compatibilidad de path para importar hints
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar variables del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Funcion auxiliar estandar (sin pensamiento extendido)
def get_completion(prompt: str, system_prompt="", prefill=""):
    messages = [{"role": "user", "content": prompt}]
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

# Funcion auxiliar para pensamiento extendido
def get_completion_with_thinking(prompt: str, system_prompt="", budget_tokens=2000):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=16000,
        thinking={
            "type": "enabled",
            "budget_tokens": budget_tokens
        },
        system=system_prompt,
        messages=[{"role": "user", "content": prompt}]
    )
    # Extraer bloques de pensamiento y texto
    thinking_text = ""
    response_text = ""
    for block in message.content:
        if block.type == "thinking":
            thinking_text = block.thinking
        elif block.type == "text":
            response_text = block.text
    return thinking_text, response_text

---

## Leccion

En el **Capitulo 6**, aprendimos a pedirle a Claude que "piense paso a paso" incluyendo su razonamiento de forma visible en la respuesta. Esto se conoce como **cadena de pensamiento** (Chain of Thought) y funciona porque forzamos a Claude a descomponer un problema antes de dar su respuesta final.

**Pensamiento Extendido** (Extended Thinking) lleva esta idea un paso mas alla. En vez de pedirle a Claude que escriba su pensamiento como parte de la respuesta visible, la API de Anthropic tiene un parametro nativo `thinking` que permite a Claude **razonar internamente** antes de responder.

### Diferencias clave entre Cadena de Pensamiento (Cap. 6) y Pensamiento Extendido

| Aspecto | Cadena de Pensamiento (Cap. 6) | Pensamiento Extendido (Cap. 10) |
|---------|-------------------------------|----------------------------------|
| Como se activa | Pidiendo en el prompt "piensa paso a paso" | Parametro `thinking` en la API |
| Pensamiento visible | Si, en la respuesta | Separado en un bloque `thinking` |
| Control | Manual, via instrucciones | Nativo, con `budget_tokens` |
| Calidad de razonamiento | Buena | Superior para problemas complejos |

### Como usar Pensamiento Extendido

Para activar el pensamiento extendido, se agrega el parametro `thinking` a la llamada de la API:

```python
message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=16000,
    thinking={
        "type": "enabled",
        "budget_tokens": 2000
    },
    messages=[{"role": "user", "content": prompt}]
)
```

El parametro `budget_tokens` controla cuantos tokens puede usar Claude para pensar. Un presupuesto mayor permite un razonamiento mas profundo.

### Restricciones importantes

1. **No se puede usar `temperature=0.0`**: El pensamiento extendido requiere que se omita el parametro temperature o que se establezca en 1.0
2. **No se puede usar pre-llenado (prefill)**: No puedes poner palabras en la boca de Claude con un mensaje de assistant cuando usas thinking
3. **`budget_tokens` minimo**: Debe ser al menos 1024
4. **`budget_tokens` < `max_tokens`**: El presupuesto de pensamiento debe ser menor que el maximo de tokens

### Demo 1: Comparacion con y sin Pensamiento Extendido

Veamos como Claude maneja un problema logico complejo **sin** y **con** pensamiento extendido.

**El problema**: Jack is looking at Anne, but Anne is looking at George. Jack is married, but George is not. Is a married person looking at an unmarried person?

In [ ]:
# Sin pensamiento extendido
PROMPT = "Jack is looking at Anne, but Anne is looking at George. Jack is married, but George is not. Is a married person looking at an unmarried person? Answer only Yes, No, or Cannot be determined."

response = get_completion(PROMPT)
print("=== Sin Pensamiento Extendido ===")
print(f"Respuesta: {response}")

In [ ]:
# Con pensamiento extendido
PROMPT = "Jack is looking at Anne, but Anne is looking at George. Jack is married, but George is not. Is a married person looking at an unmarried person? Answer only Yes, No, or Cannot be determined."

thinking, response = get_completion_with_thinking(PROMPT, budget_tokens=2000)

print("=== Con Pensamiento Extendido ===")
print(f"\nPensamiento de Claude (interno):\n{thinking}")
print(f"\n{'='*50}")
print(f"\nRespuesta final: {response}")

**Analisis**: La respuesta correcta es **Yes** (Si). El truco esta en considerar los dos casos posibles para Anne:
- Si Anne esta casada: ella (casada) esta mirando a George (no casado) -> Si
- Si Anne no esta casada: Jack (casado) esta mirando a Anne (no casada) -> Si

En ambos casos, una persona casada esta mirando a una persona no casada. Sin pensamiento extendido, Claude a menudo responde "Cannot be determined" porque no considera ambos escenarios sistematicamente.

### Demo 2: Efecto del presupuesto de tokens (budget_tokens)

El parametro `budget_tokens` controla la profundidad del razonamiento. Veamos como diferentes presupuestos afectan la calidad de respuesta para un problema matematico.

In [ ]:
PROMPT = """A farmer has 17 sheep. All but 9 die. How many sheep does the farmer have left? Explain your reasoning step by step, then give the final answer."""

# Probar con diferentes presupuestos
for budget in [1024, 3000, 6000]:
    thinking, response = get_completion_with_thinking(PROMPT, budget_tokens=budget)
    print(f"\n{'='*60}")
    print(f"Budget: {budget} tokens")
    print(f"Longitud del pensamiento: {len(thinking)} caracteres")
    print(f"Respuesta: {response[:200]}...")

### Cuando usar Pensamiento Extendido

**Usa Pensamiento Extendido cuando:**
- Problemas de matematicas o logica complejos
- Analisis de codigo con multiples dependencias
- Tareas que requieren razonamiento multi-paso
- Cuando la precision importa mas que la velocidad o el costo

**NO uses Pensamiento Extendido cuando:**
- Preguntas factuales simples ("Cual es la capital de Francia?")
- Escritura creativa sin restricciones
- Cuando la latencia y el costo son prioritarios
- Cuando necesitas `temperature=0.0` para resultados deterministicos
- Cuando necesitas pre-llenado (prefill) de la respuesta

---

## Ejercicios
- [Ejercicio 10.1 - Pensamiento Extendido vs Normal](#ejercicio-101---pensamiento-extendido-vs-normal)
- [Ejercicio 10.2 - Optimizar el Presupuesto](#ejercicio-102---optimizar-el-presupuesto)
- [Ejercicio 10.3 - Combinar con System Prompts](#ejercicio-103---combinar-con-system-prompts)

### Ejercicio 10.1 - Pensamiento Extendido vs Normal

Usa el siguiente problema de logica. Primero resuelvelo **sin** pensamiento extendido usando `get_completion()`, y luego **con** pensamiento extendido usando `get_completion_with_thinking()`. Compara las respuestas.

**Problema**: If it takes 5 machines 5 minutes to make 5 widgets, how long would it take 100 machines to make 100 widgets?

In [ ]:
# Prompt - modifica si quieres
PROMPT = "If it takes 5 machines 5 minutes to make 5 widgets, how long would it take 100 machines to make 100 widgets? Give only the numerical answer in minutes."

# Sin pensamiento extendido
response_normal = get_completion(PROMPT)
print("Sin thinking:", response_normal)

# Con pensamiento extendido
thinking, response_thinking = get_completion_with_thinking(PROMPT, budget_tokens=2000)
print("\nCon thinking:", response_thinking)
print("\nPensamiento:", thinking[:500])

# Funcion de calificacion
def grade_exercise(normal_resp, thinking_resp):
    return "5" in thinking_resp

print("\n--------------------------- CALIFICACION ---------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response_normal, response_thinking))

La respuesta correcta es **5 minutos**. Cada maquina produce 1 widget en 5 minutos, asi que 100 maquinas producen 100 widgets en los mismos 5 minutos.

Si quieres una pista, ejecuta la celda siguiente!

In [ ]:
from hints import exercise_10_1_hint; print(exercise_10_1_hint)

### Ejercicio 10.2 - Optimizar el Presupuesto

Para el siguiente problema, encuentra el `budget_tokens` **minimo** que produce una respuesta correcta. Empieza con 1024 y ve subiendo.

**Problema**: A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much does the ball cost?

In [ ]:
PROMPT = "A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much does the ball cost? Give only the dollar amount."

# Experimenta con diferentes budget_tokens - cambia este valor
BUDGET = 1024  # Intenta: 1024, 2000, 3000, 4000

thinking, response = get_completion_with_thinking(PROMPT, budget_tokens=BUDGET)

print(f"Budget: {BUDGET} tokens")
print(f"Respuesta: {response}")
print(f"Longitud pensamiento: {len(thinking)} chars")

# Funcion de calificacion
def grade_exercise(text):
    return bool(re.search(r'0\.05|5 cents|five cents', text.lower()))

print("\n--------------------------- CALIFICACION ---------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

La respuesta correcta es **$0.05** (5 centavos). Muchas personas (y LLMs sin pensar) responden $0.10 intuitivamente.

Si quieres una pista, ejecuta la celda siguiente!

In [ ]:
from hints import exercise_10_2_hint; print(exercise_10_2_hint)

### Ejercicio 10.3 - Combinar con System Prompts

Combina un system prompt detallado con pensamiento extendido para analizar el siguiente fragmento de codigo y encontrar todos los bugs.

In [ ]:
SYSTEM_PROMPT = """You are an expert code reviewer. Analyze the provided code and:
1. List ALL bugs you find
2. Explain why each is a bug
3. Provide the corrected code
Be thorough and systematic."""

CODE = """
def calculate_average(numbers):
    total = 0
    for i in range(len(numbers)):
        total += numbers[i]
    average = total / len(numbers)
    return average

def find_max(numbers):
    max_val = 0
    for num in numbers:
        if num > max_val:
            max_val = num
    return max_val

def remove_duplicates(lst):
    for item in lst:
        if lst.count(item) > 1:
            lst.remove(item)
    return lst
"""

PROMPT = f"Review this Python code and find all bugs:\n{CODE}"

thinking, response = get_completion_with_thinking(PROMPT, system_prompt=SYSTEM_PROMPT, budget_tokens=4000)

print("=== Analisis de Claude ===")
print(response)

# Funcion de calificacion
def grade_exercise(text):
    # Buscar que identifique al menos 2 de los 3 bugs principales
    bugs_found = 0
    if re.search(r'empty|zero|division|len.*0|ZeroDivision', text, re.IGNORECASE):
        bugs_found += 1
    if re.search(r'negative|max.*0|initial', text, re.IGNORECASE):
        bugs_found += 1
    if re.search(r'modify.*iterating|mutating|remove.*during|concurrent', text, re.IGNORECASE):
        bugs_found += 1
    return bugs_found >= 2

print("\n--------------------------- CALIFICACION ---------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Los 3 bugs principales son:
1. `calculate_average`: division por cero si la lista esta vacia
2. `find_max`: inicializa `max_val = 0`, falla con numeros negativos
3. `remove_duplicates`: modifica la lista mientras itera sobre ella

Si quieres una pista, ejecuta la celda siguiente!

In [ ]:
from hints import exercise_10_3_hint; print(exercise_10_3_hint)

### Felicidades!

Si has resuelto todos los ejercicios, has aprendido a usar el pensamiento extendido de Claude para mejorar la calidad de sus respuestas en problemas complejos. Continua al Capitulo 11 para aprender sobre System Prompts avanzados!

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de esta leccion y modifiques los prompts para ver como puede afectar las respuestas de Claude.

In [ ]:
# Experimenta aqui con pensamiento extendido
PROMPT = "Your prompt here"

thinking, response = get_completion_with_thinking(PROMPT, budget_tokens=2000)
print("Pensamiento:", thinking[:500])
print("\nRespuesta:", response)

In [ ]:
# Compara con y sin thinking
PROMPT = "Your prompt here"

print("Sin thinking:")
print(get_completion(PROMPT))

print("\nCon thinking:")
thinking, response = get_completion_with_thinking(PROMPT)
print(response)